In [ ]:
pip install -r requirements.txt

: 

In [ ]:
pip install matplotlib

In [ ]:
pip install sklearn

In [ ]:
pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, ConvexHull
from sklearn.metrics import silhouette_score
from folium.plugins import HeatMap

In [ ]:
df=pd.read_csv('Customer_data.csv')

In [ ]:
df['order_count'] = df['order_count'] + 1  # Increment all values by 1
df['order_count'].fillna(1, inplace=True)  # Fill NaN values with 1

In [ ]:
# Remove rows with missing latitude or longitude values
df = df.dropna(subset=['Delivery_location_latitude', 'Delivery_location_longitude'])

In [ ]:


# 1️⃣ Load Customer Demand Data
df=pd.read_csv("Customer_data.csv")
# 2️⃣ Preprocess Data
df['order_count'] = df['order_count'].fillna(0) + 1  # Fill NaN with 0, then add 1

# Drop rows with missing location data
df = df.dropna(subset=['Delivery_location_latitude', 'Delivery_location_longitude'])

# Extract customer locations & demand weights
customer_locations = df[['Delivery_location_latitude', 'Delivery_location_longitude']].values
customer_weights = df['order_count'].values  # Demand-based weighting

# 3️⃣ Determine Optimal k for Clustering
sse = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(customer_locations, sample_weight=customer_weights)
    sse.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(customer_locations, labels))

# Plot Elbow & Silhouette Method
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(K_range, sse, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Dark Stores (k)')
plt.ylabel('SSE (Inertia)')
plt.title('Elbow Method')

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, marker='o', linestyle='--', color='g')
plt.xlabel('Number of Dark Stores (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')

plt.tight_layout()
plt.show()

# Select optimal k (Choose highest silhouette score OR elbow point)
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"Optimal number of dark stores (k): {optimal_k}")

# optimal_k = 17

# 4️⃣ Apply K-Means Clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(customer_locations, sample_weight=customer_weights)

# Get cluster centers (Suggested dark store locations)
dark_store_locations = kmeans.cluster_centers_

# 5️⃣ Compute Voronoi Diagram for store regions
vor = Voronoi(dark_store_locations)

# Function to get Voronoi regions as polygons
def voronoi_regions(vor):
    polygons = []
    for region in vor.regions:
        if not region or -1 in region:
            continue
        polygon = [vor.vertices[i] for i in region]
        polygons.append(Polygon(polygon))
    return polygons

voronoi_polygons = voronoi_regions(vor)
gdf_voronoi = gpd.GeoDataFrame(geometry=voronoi_polygons)

# 6️⃣ Create Pune Map
city_center = [df['Delivery_location_latitude'].mean(), df['Delivery_location_longitude'].mean()]
m = folium.Map(location=city_center, zoom_start=12)

# 7️⃣ Add Customer Density as Heatmap
heat_data = [[lat, lon, weight] for (lat, lon), weight in zip(customer_locations, customer_weights)]
HeatMap(heat_data, radius=12, blur=8, max_zoom=15).add_to(m)

# 8️⃣ Plot Voronoi Regions
for polygon in gdf_voronoi.geometry:
    geojson = gpd.GeoSeries([polygon]).__geo_interface__
    folium.GeoJson(geojson, style_function=lambda x: {'fillColor': 'lightblue', 'color': 'blue', 'weight': 1}).add_to(m)

# 9️⃣ Plot Dark Store Locations
for idx, loc in enumerate(dark_store_locations):
    folium.Marker(
        location=[loc[0], loc[1]],
        popup=f"Dark Store {idx+1}",
        icon=folium.Icon(color='red', icon='home')
    ).add_to(m)

# 🔟 Draw Cluster Boundaries with Dashed Lines
for cluster in range(optimal_k):
    cluster_points = df[df['cluster'] == cluster][['Delivery_location_latitude', 'Delivery_location_longitude']].values
    if len(cluster_points) > 2:
        hull = ConvexHull(cluster_points)
        hull_points = [cluster_points[i] for i in hull.vertices] + [cluster_points[hull.vertices[0]]]
        hull_line = LineString(hull_points)
        folium.PolyLine(
            locations=[(p[0], p[1]) for p in hull_line.coords],
            color='black', dash_array='5,5', weight=2
        ).add_to(m)

# Display the Map
m

: 

In [ ]:


# 1️⃣ Load Customer Demand Data
df=pd.read_csv("Customer_data.csv")
# 2️⃣ Preprocess Data
df['order_count'] = df['order_count'].fillna(0) + 1  # Fill NaN with 0, then add 1

# Drop rows with missing location data
df = df.dropna(subset=['Delivery_location_latitude', 'Delivery_location_longitude'])

# Extract customer locations & demand weights
customer_locations = df[['Delivery_location_latitude', 'Delivery_location_longitude']].values
customer_weights = df['order_count'].values  # Demand-based weighting

# 3️⃣ Determine Optimal k for Clustering
sse = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(customer_locations, sample_weight=customer_weights)
    sse.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(customer_locations, labels))

# Plot Elbow & Silhouette Method
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(K_range, sse, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Dark Stores (k)')
plt.ylabel('SSE (Inertia)')
plt.title('Elbow Method')

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, marker='o', linestyle='--', color='g')
plt.xlabel('Number of Dark Stores (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')

plt.tight_layout()
plt.show()

# Select optimal k (Choose highest silhouette score OR elbow point)
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"Optimal number of dark stores (k): {optimal_k}")

# optimal_k = 17

# 4️⃣ Apply K-Means Clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(customer_locations, sample_weight=customer_weights)

# Get cluster centers (Suggested dark store locations)
dark_store_locations = kmeans.cluster_centers_

# 5️⃣ Compute Voronoi Diagram for store regions
vor = Voronoi(dark_store_locations)

# Function to get Voronoi regions as polygons
def voronoi_regions(vor):
    polygons = []
    for region in vor.regions:
        if not region or -1 in region:
            continue
        polygon = [vor.vertices[i] for i in region]
        polygons.append(Polygon(polygon))
    return polygons

voronoi_polygons = voronoi_regions(vor)
gdf_voronoi = gpd.GeoDataFrame(geometry=voronoi_polygons)

# 6️⃣ Create Pune Map
city_center = [df['Delivery_location_latitude'].mean(), df['Delivery_location_longitude'].mean()]
m = folium.Map(location=city_center, zoom_start=12)

# 7️⃣ Add Customer Density as Heatmap
heat_data = [[lat, lon, weight] for (lat, lon), weight in zip(customer_locations, customer_weights)]
HeatMap(heat_data, radius=12, blur=8, max_zoom=15).add_to(m)

# 8️⃣ Plot Voronoi Regions
for polygon in gdf_voronoi.geometry:
    geojson = gpd.GeoSeries([polygon]).__geo_interface__
    folium.GeoJson(geojson, style_function=lambda x: {'fillColor': 'lightblue', 'color': 'blue', 'weight': 1}).add_to(m)

# 9️⃣ Plot Dark Store Locations
for idx, loc in enumerate(dark_store_locations):
    folium.Marker(
        location=[loc[0], loc[1]],
        popup=f"Dark Store {idx+1}",
        icon=folium.Icon(color='red', icon='home')
    ).add_to(m)

# 🔟 Draw Cluster Boundaries with Dashed Lines
for cluster in range(optimal_k):
    cluster_points = df[df['cluster'] == cluster][['Delivery_location_latitude', 'Delivery_location_longitude']].values
    if len(cluster_points) > 2:
        hull = ConvexHull(cluster_points)
        hull_points = [cluster_points[i] for i in hull.vertices] + [cluster_points[hull.vertices[0]]]
        hull_line = LineString(hull_points)
        folium.PolyLine(
            locations=[(p[0], p[1]) for p in hull_line.coords],
            color='black', dash_array='5,5', weight=2
        ).add_to(m)

# Display the Map
m